In [5]:
# END-TO-END with ROBUST FILE DISCOVERY
# Builds data/processed/team_season_features_v2_clean.csv for 1985–2025 (2025 cutoff June 22)

import pandas as pd
import numpy as np
from pathlib import Path

# ---------- Paths ----------
BASE = Path(".").resolve()
RAW  = BASE / "data" / "raw"
PROC = BASE / "data" / "processed"
PROC.mkdir(parents=True, exist_ok=True)

# ---------- Helpers ----------
def pick_one(paths, label):
    """Choose the first path from a list and print all candidates for transparency."""
    if not paths:
        return None
    print(f"\n[{label}] candidates found ({len(paths)}):")
    for i, p in enumerate(paths[:12], 1):
        print(f"  {i:>2}. {p}")
    return paths[0]

def find_csv(label_patterns):
    """Search common locations and recursively for any of the patterns."""
    # Common locations first
    common_dirs = [
        RAW,
        BASE / "data",
        BASE,
        BASE.parent,                  # one level up
        BASE.parent.parent,           # two levels up
    ]
    candidates = []
    for d in common_dirs:
        if d.exists():
            for pat in label_patterns:
                candidates += list(d.glob(pat))
    # Fallback: deep recursive search
    if not candidates:
        for pat in label_patterns:
            candidates += list(BASE.rglob(pat))
    # De-dup and sort by path length (prefer shorter, likely correct)
    candidates = sorted(set(candidates), key=lambda p: (len(str(p)), str(p)))
    return candidates

# ---------- Try to find the files (flexible patterns) ----------
team_stats_candidates = find_csv([
    "TeamStatistics.csv",
    "team_statistics.csv",
    "*Team*Stat*.csv",
])

games_candidates = find_csv([
    "Games.csv",
    "games.csv",
    "*Game*.csv",
])

ts_path = pick_one(team_stats_candidates, "TeamStatistics")
gm_path = pick_one(games_candidates,     "Games")

if ts_path is None or gm_path is None:
    print("\nWorking dir:", BASE)
    print("Searched under:", RAW, "and the whole repo.")
    raise FileNotFoundError(
        "Could not find TeamStatistics.csv or Games.csv. "
        "Fix: place them in data/raw/ or tell me the exact paths/filenames."
    )

print(f"\nUsing:\n  TeamStatistics -> {ts_path}\n  Games         -> {gm_path}")

# ---------- Load ----------
ts = pd.read_csv(ts_path, low_memory=False)
games = pd.read_csv(gm_path, low_memory=False)

# ---------- Season & date handling ----------
def to_season(dt_ser):
    dt = pd.to_datetime(dt_ser.astype(str), utc=True, errors="coerce").dt.tz_convert("US/Eastern")
    return dt.dt.year + (dt.dt.month >= 8)

ts["season"] = to_season(ts["gameDate"])
ts["date"]   = pd.to_datetime(ts["gameDate"], utc=True, errors="coerce").dt.tz_convert(None)

# Keep seasons 1985–2025
ts = ts[ts["season"].between(1985, 2025)].copy()

# Apply Finals cutoff for 2025 (include only games on/before 2025-06-22)
CUTOFF_2025 = pd.to_datetime("2025-06-22")
ts = ts[~((ts["season"] == 2025) & (ts["date"] > CUTOFF_2025))]

# ---------- NBA 30 filtering ----------
NBA30 = {
    "Hawks","Celtics","Nets","Hornets","Bulls","Cavaliers","Mavericks","Nuggets","Pistons","Warriors",
    "Rockets","Pacers","Clippers","Lakers","Grizzlies","Heat","Bucks","Timberwolves","Pelicans","Knicks",
    "Thunder","Magic","76ers","Suns","Trail Blazers","Kings","Spurs","Raptors","Jazz","Wizards"
}
ts["team"] = ts["teamName"].astype(str).str.strip()
ts["opp"]  = ts["opponentTeamName"].astype(str).str.strip()
ts = ts[ts["team"].isin(NBA30) & ts["opp"].isin(NBA30)].copy()

# ---------- Ensure numeric ----------
num_cols = [
    "teamScore","opponentScore","assists","reboundsOffensive","reboundsDefensive","reboundsTotal",
    "turnovers","steals","blocks","win"
]
for c in num_cols:
    if c in ts.columns:
        ts[c] = pd.to_numeric(ts[c], errors="coerce")

# ---------- Bring opponent stats (for differentials/ratios) ----------
opp = ts[[
    "gameId","teamId","reboundsDefensive","reboundsOffensive","reboundsTotal","turnovers",
    "assists","steals","blocks","teamScore"
]].rename(columns={
    "teamId":"opp_teamId",
    "reboundsDefensive":"opp_reb_def",
    "reboundsOffensive":"opp_reb_off",
    "reboundsTotal":"opp_reb_total",
    "turnovers":"opp_to",
    "assists":"opp_ast",
    "steals":"opp_stl",
    "blocks":"opp_blk",
    "teamScore":"opp_pts_for",
})
ts = ts.merge(
    opp,
    left_on=["gameId","opponentTeamId"],
    right_on=["gameId","opp_teamId"],
    how="left"
)

# ---------- Per-game derived metrics ----------
eps = 1e-9
ts["point_diff"]                = ts["teamScore"] - ts["opponentScore"]
ts["reb_margin_game"]           = ts["reboundsTotal"] - ts["opp_reb_total"]
ts["off_reb_rate_game"]         = ts["reboundsOffensive"] / (ts["reboundsOffensive"] + ts["opp_reb_def"] + eps)
ts["assist_turnover_ratio_game"]= ts["assists"] / (ts["turnovers"] + eps)
ts["steal_turnover_ratio_game"] = ts["steals"]  / (ts["turnovers"] + eps)

# ---------- Strength of schedule (season-level opp win%) ----------
season_win = (
    ts.groupby(["season","team"], as_index=False)
      .agg(games=("win","size"), wins=("win","sum"))
)
season_win["win_pct"] = season_win["wins"]/season_win["games"]
opp_win = season_win.rename(columns={"team":"opp","win_pct":"opp_win_pct"})[["season","opp","opp_win_pct"]]
ts = ts.merge(opp_win, on=["season","opp"], how="left")

# ---------- Aggregate to team-season features ----------
agg_mean = ts.groupby(["season","team"]).agg(
    games=("win","size"),
    wins=("win","sum"),
    win_pct=("win","mean"),
    avg_pts_for=("teamScore","mean"),
    avg_pts_against=("opponentScore","mean"),
    avg_point_diff=("point_diff","mean"),
    avg_assists=("assists","mean"),
    avg_reb_off=("reboundsOffensive","mean"),
    avg_reb_def=("reboundsDefensive","mean"),
    avg_reb_total=("reboundsTotal","mean"),
    avg_turnovers=("turnovers","mean"),
    avg_steals=("steals","mean"),
    avg_blocks=("blocks","mean"),
    sos=("opp_win_pct","mean"),
    avg_reb_margin=("reb_margin_game","mean"),
    avg_off_reb_rate=("off_reb_rate_game","mean"),
    avg_ast_to=("assist_turnover_ratio_game","mean"),
    avg_stl_to=("steal_turnover_ratio_game","mean"),
).reset_index()

# Sum-based robust ratios
agg_sum = ts.groupby(["season","team"]).agg(
    sum_assists=("assists","sum"),
    sum_turnovers=("turnovers","sum"),
    sum_steals=("steals","sum")
).reset_index()

features = agg_mean.merge(agg_sum, on=["season","team"], how="left")
features["assist_turnover_ratio"] = features["sum_assists"] / (features["sum_turnovers"] + eps)
features["steal_turnover_ratio"]  = features["sum_steals"]  / (features["sum_turnovers"] + eps)
features = features.drop(columns=["sum_assists","sum_turnovers","sum_steals"])

# ---------- Champion labels (1985–2025); 2025 = Thunder ----------
champ_map = {
    1985:"Lakers",1986:"Celtics",1987:"Lakers",1988:"Lakers",1989:"Pistons",
    1990:"Pistons",1991:"Bulls",1992:"Bulls",1993:"Bulls",1994:"Rockets",
    1995:"Rockets",1996:"Bulls",1997:"Bulls",1998:"Bulls",1999:"Spurs",
    2000:"Lakers",2001:"Lakers",2002:"Lakers",2003:"Spurs",2004:"Pistons",
    2005:"Spurs",2006:"Heat",2007:"Spurs",2008:"Celtics",2009:"Lakers",
    2010:"Lakers",2011:"Mavericks",2012:"Heat",2013:"Heat",2014:"Spurs",
    2015:"Warriors",2016:"Cavaliers",2017:"Warriors",2018:"Warriors",2019:"Raptors",
    2020:"Lakers",2021:"Bucks",2022:"Warriors",2023:"Nuggets",2024:"Celtics",2025:"Thunder"
}
features["champion"] = 0
for yr, tm in champ_map.items():
    features.loc[(features["season"]==yr) & (features["team"]==tm), "champion"] = 1

# Remove sparse columns if they slipped in
for c in ["avg_fastbreak_points","avg_paint_points","avg_second_chance_points","avg_bench_points"]:
    if c in features.columns:
        features = features.drop(columns=[c])

# ---------- Save ----------
out_path = PROC / "team_season_features_v2_clean.csv"
features = features.sort_values(["season","team"]).reset_index(drop=True)
features.to_csv(out_path, index=False)

print("\nWROTE:", out_path)
print("SHAPE:", features.shape)
print(features.head(3))
print(features.tail(3))
print("\nChampions check (last 10):")
print(features.loc[features["champion"]==1, ["season","team"]].tail(10))




[TeamStatistics] candidates found (1):
   1. /Users/kennethchen/CS506-FALL-2025/CS506FinalProject/notebooks/data/raw/TeamStatistics.csv

[Games] candidates found (2):
   1. /Users/kennethchen/CS506-FALL-2025/CS506FinalProject/notebooks/data/raw/Games.csv
   2. /Users/kennethchen/CS506-FALL-2025/CS506FinalProject/notebooks/data/raw/games.csv

Using:
  TeamStatistics -> /Users/kennethchen/CS506-FALL-2025/CS506FinalProject/notebooks/data/raw/TeamStatistics.csv
  Games         -> /Users/kennethchen/CS506-FALL-2025/CS506FinalProject/notebooks/data/raw/Games.csv

WROTE: /Users/kennethchen/CS506-FALL-2025/CS506FinalProject/notebooks/data/processed/team_season_features_v2_clean.csv
SHAPE: (1123, 23)
   season   team  games  wins   win_pct  avg_pts_for  avg_pts_against  \
0    1985  76ers     83    57  0.686747   112.457831       108.638554   
1    1985  Bucks     82    56  0.682927   111.451220       105.768293   
2    1985  Bulls     79    34  0.430380   109.000000       110.759494   

   av